In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
!grep Clapping ../dataset/class_labels_indices.csv

63,/m/0l15bq,"Clapping"


In [5]:
class_label_index = !grep Clapping ../dataset/class_labels_indices.csv

In [6]:
print(class_label_index[0].split(",")[1])

/m/0l15bq


In [7]:
!grep /m/0l15bq ../dataset/balanced_train_segments.csv |head

0FMdORf5iGs, 30.000, 40.000, "/m/04rlf,/m/081rb,/m/09x0r,/m/0l15bq"
1IxBagCJeZc, 150.000, 160.000, "/m/01j3sz,/m/09x0r,/m/0l15bq"
1_DouJRW3PM, 30.000, 40.000, "/m/028ght,/m/09x0r,/m/0l15bq"
2y9ikTsTsl0, 30.000, 40.000, "/m/028ght,/m/09x0r,/m/0l15bq"
3PliaLqMSqg, 30.000, 40.000, "/m/028ght,/m/09x0r,/m/0l15bq"
3ixOXsKUufM, 30.000, 40.000, "/m/0l15bq"
4mOTOTJLv5U, 0.000, 10.000, "/m/09x0r,/m/0l15bq,/m/0ytgt"
7Ep2a7_sbmc, 260.000, 270.000, "/m/09x0r,/m/0l15bq"
7SpYywlGPyM, 30.000, 40.000, "/m/09x0r,/m/0k65p,/m/0l15bq,/m/0ytgt"
AiGF0850kT8, 6.000, 16.000, "/m/04rlf,/m/0l15bq"


In [8]:
examples = !grep /m/0l15bq ../dataset/balanced_train_segments.csv | head -2 | cut -c -11
print(examples)

['0FMdORf5iGs', '1IxBagCJeZc']


In [9]:
tfrecord_prefixes = set([i[:2] for i in examples])

In [10]:

tfrecord_filenames = ["../dataset/bal_train/" + i + ".tfrecord" for i in tfrecord_prefixes]

In [11]:

audio_embeddings_dict = {}
audio_labels_dict = {}
#all_tfrecord_filenames = glob.glob("bal_train/" + example[:2] + ".tfrecord")

# Load embeddings
sess = tf.Session() 
for tfrecord in tfrecord_filenames: 
  for example in tf.python_io.tf_record_iterator(tfrecord):
    tf_example = tf.train.Example.FromString(example)
    vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding='UTF-8')
    if vid_id in examples:
      example_label = list(np.asarray(tf_example.features.feature['labels'].int64_list.value))
      tf_seq_example = tf.train.SequenceExample.FromString(example)
      n_frames = len(tf_seq_example.feature_lists.feature_list['audio_embedding'].feature)
    
      audio_frame = []
      for i in range(n_frames):
        audio_frame.append(tf.cast(tf.decode_raw(
             tf_seq_example.feature_lists.feature_list['audio_embedding'].feature[i].bytes_list.value[0],tf.uint8)
            ,tf.float32).eval(session=sess))

      audio_embeddings_dict[vid_id] = audio_frame
      audio_labels_dict[vid_id] = example_label

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [31]:
print (audio_embeddings_dict)
print (audio_labels_dict)

{'1IxBagCJeZc': [array([125., 119., 225., 141., 161., 146.,  89.,   0., 235., 206.,  45.,
        36.,  90., 131.,  95.,  98., 120.,  76., 253.,  74., 154., 151.,
       116., 125.,  52.,  60.,  89., 206., 105.,  69.,  84.,  40., 108.,
       165., 123., 200., 197., 110., 176.,  40., 120., 116., 177., 137.,
       170., 142., 128., 184.,  49.,  68., 149., 127.,   2., 251., 126.,
        87., 146., 183., 145., 149.,  12.,  46., 176.,  35., 181., 205.,
        88.,  88., 197., 106., 201., 102., 175., 236., 222., 114., 255.,
       238., 188.,  81., 204., 160., 176.,  22.,  78.,  52., 197., 209.,
       146.,  53.,  92., 122., 166.,  85.,  98., 232., 200.,  65., 246.,
        69., 126., 210., 245., 160., 255.,  36., 151.,  71., 169., 102.,
        27., 128.,   2.,   4.,  20., 145., 221., 255.,  26., 130.,   0.,
        43.,  44., 121., 150.,   0.,  73.,  43.], dtype=float32), array([ 89.,  59., 210., 136., 100., 153.,  43.,  62., 210., 146.,   0.,
        69., 175.,  98.,  96., 145.,  73.

In [13]:
import soundfile as sf

In [14]:
audio_teste = audio_embeddings_dict['1IxBagCJeZc'][0]
print(audio_teste)

[125. 119. 225. 141. 161. 146.  89.   0. 235. 206.  45.  36.  90. 131.
  95.  98. 120.  76. 253.  74. 154. 151. 116. 125.  52.  60.  89. 206.
 105.  69.  84.  40. 108. 165. 123. 200. 197. 110. 176.  40. 120. 116.
 177. 137. 170. 142. 128. 184.  49.  68. 149. 127.   2. 251. 126.  87.
 146. 183. 145. 149.  12.  46. 176.  35. 181. 205.  88.  88. 197. 106.
 201. 102. 175. 236. 222. 114. 255. 238. 188.  81. 204. 160. 176.  22.
  78.  52. 197. 209. 146.  53.  92. 122. 166.  85.  98. 232. 200.  65.
 246.  69. 126. 210. 245. 160. 255.  36. 151.  71. 169. 102.  27. 128.
   2.   4.  20. 145. 221. 255.  26. 130.   0.  43.  44. 121. 150.   0.
  73.  43.]


In [37]:
#sf.write('out.wav', audio_teste, 44100)
audio_final = []

for i in range(len(audio_embeddings_dict['1IxBagCJeZc'])):
    for j in audio_embeddings_dict['1IxBagCJeZc'][i]:
        audio_final.append(j)
        a += 1

#print(audio_final)
sf.write('out.wav', audio_final, 32000)

128
256
384
512
640
768
896
1024
1152
1280
